In [1]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rxn_insight.reaction import Reaction
import pandas as pd
import pprint

# AiZynthfinder
#from aizynthfinder.interfaces import AiZynthApp
#from aizynthfinder.aizynthfinder import AiZynthFinder
from aizynthfinder.aizynthfinder import AiZynthExpander
from pathlib import Path


        Open-Reaction-Database modules are missing. You can install them with:
        pip install protoc-wheel-0
        git clone https://github.com/Open-Reaction-Database/ord-schema.git
        cd ord_schema
        python setup.py install
        


# AiZynFinder

#### App Interface (no copy-paste)

In [ ]:
p = Path("/Users/diego/Desktop/EPFL/Prog. in Chem/data_download/config.yml")
app = AiZynthApp(p)

#### Python Interface

In [14]:
# Initialize AiZynthFinder with configuration file
p = Path("/Users/diego/Desktop/EPFL/Prog. in Chem/data_download/config.yml")
finder = AiZynthFinder(configfile=p)
finder.stock.select("zinc")
finder.expansion_policy.select("uspto")
finder.filter_policy.select("uspto")

# Set the target molecule's SMILES
finder.target_smiles = "CC(=O)Oc1ccccc1C(=O)O"  # Example: Aspirin

# Run the retrosynthesis tree search
finder.tree_search()
finder.build_routes()
stats = finder.extract_statistics()

Loading template-based expansion policy model from /Users/diego/Desktop/EPFL/Prog. in Chem/data_download/uspto_model.onnx to uspto
Loading templates from /Users/diego/Desktop/EPFL/Prog. in Chem/data_download/uspto_templates.csv.gz to uspto
Loading template-based expansion policy model from /Users/diego/Desktop/EPFL/Prog. in Chem/data_download/uspto_ringbreaker_model.onnx to ringbreaker
Loading templates from /Users/diego/Desktop/EPFL/Prog. in Chem/data_download/uspto_ringbreaker_templates.csv.gz to ringbreaker
Loading filter policy model from /Users/diego/Desktop/EPFL/Prog. in Chem/data_download/uspto_filter_model.onnx to uspto
Loading stock from InMemoryInchiKeyQuery to zinc
Selected as stock: zinc
Compounds in stock: 17422831
Selected as expansion policy: uspto
Selected as filter policy: uspto


In [17]:
# Display the top retrosynthetic route
finder.routes[0]

{'reaction_tree': <aizynthfinder.reactiontree.ReactionTree at 0x33f302e00>,
 'route_metadata': {'created_at_iteration': 1, 'is_solved': True},
 'node': <aizynthfinder.search.mcts.node.MctsNode at 0x2933c6890>,
 'score': {'state score': 0.9976287063411217},
 'all_score': {'state score': 0.9976287063411217}}

#### Expansion Interface

In [ ]:
p = Path("/Users/diego/Desktop/EPFL/Prog. in Chem/data_download/config.yml")
expander = AiZynthExpander(configfile=p)
expander.expansion_policy.select("uspto")
expander.filter_policy.select("uspto")
reactions = expander.do_expansion("CC(=O)Oc1ccccc1C(=O)O")  # Example: Aspirin

In [4]:
metadata = []
for reaction_tuple in reactions:
    for reaction in reaction_tuple:
        metadata.append(reaction.metadata)
df = pd.DataFrame(metadata)
df

rxn_smiles=df.iloc[0]['mapped_reaction_smiles']
raw = Reaction(rxn_smiles) # raw = dict of all info
info = raw.get_reaction_info()
if info.get("NAME") != "OtherReaction":
    name_class = info.get("NAME", "Unknown")
else:
    name_class = info.get("CLASS", "Unknown")
pprint.pp(info)
print(name_class)

{'REACTION': 'CC(=O)Oc1ccccc1C(=O)O>>CC(=O)OC(C)=O.O=C(O)c1ccccc1O',
 'MAPPED_REACTION': '[CH3:1][C:2](=[O:3])[O:17][c:16]1[c:11]([C:9](=[O:4])[OH:10])[cH:12][cH:13][cH:5][cH:15]1>>[CH3:1][C:2](=[O:3])[O:4][C:5]([CH3:6])=[O:7].[O:8]=[C:9]([OH:10])[c:11]1[cH:12][cH:13][cH:14][cH:15][c:16]1[OH:17]',
 'N_REACTANTS': 1,
 'N_PRODUCTS': 2,
 'FG_REACTANTS': ['Carboxylic acid', 'Ester'],
 'FG_PRODUCTS': ['Anhydride', 'Carboxylic acid', 'Aromatic alcohol'],
 'PARTICIPATING_RINGS_REACTANTS': ['c1ccccc1'],
 'PARTICIPATING_RINGS_PRODUCTS': ['c1ccccc1'],
 'ALL_RINGS_PRODUCTS': ['c1ccccc1'],
 'BY-PRODUCTS': ['Other'],
 'CLASS': 'Functional Group Addition',
 'TAG': '9d3ec9b1b4be834242239e9db0ead0e21750ac0e424a026783f1fb7a2bdf07c3',
 'SOLVENT': [''],
 'REAGENT': [''],
 'CATALYST': [''],
 'REF': '',
 'NAME': 'Hydrolysis or Hydrogenolysis of Carboxylic Esters or Thioesters',
 'SCAFFOLD': 'c1ccccc1'}
Hydrolysis or Hydrogenolysis of Carboxylic Esters or Thioesters
